In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [4]:
import geopandas as gpd
import pandas as pd
from siuba import *
import numpy as np

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2
import folium
import itertools
import sys
sys.path.append('./scripts/')

In [5]:
from update_vars import (analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, AM_PEAK, PM_PEAK, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD, SHARED_STOP_THRESHOLD)

In [6]:
import create_aggregate_stop_frequencies as casf

In [7]:
import importlib
importlib.reload(casf)

<module 'create_aggregate_stop_frequencies' from '/home/jovyan/data-analyses/high_quality_transit_areas/./scripts/create_aggregate_stop_frequencies.py'>

In [8]:
am_peak_hrs = list(range(AM_PEAK[0].hour, AM_PEAK[1].hour))
pm_peak_hrs = list(range(PM_PEAK[0].hour, PM_PEAK[1].hour))
both_peaks_hrs = am_peak_hrs + pm_peak_hrs

In [9]:
analysis_date

'2025-05-14'

In [10]:
stop_times = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
)

In [11]:
stop_times = casf.add_route_dir(stop_times, analysis_date)

In [12]:
st_prepped = stop_times.pipe(casf.prep_stop_times)

/opt/conda/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


## HCD Y-Branching

* combine single route frequencies with collinearity screen...
* actually, these routes need to pass _both_ a collinearity screen and a non-collinearity screen: 8+ stops shared and also 8+ stops not shared (meaningful service on all 3 legs of the Y)

In [13]:
max_arrivals_by_stop_single = st_prepped.pipe(casf.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=True)
max_arrivals_by_stop_multi = st_prepped.pipe(casf.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=False)

In [14]:
multi_only_explode = casf.get_explode_multiroute_only(max_arrivals_by_stop_single, max_arrivals_by_stop_multi, (HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD))

In [15]:
multi_only_explode.head(3)

,schedule_gtfs_dataset_key,stop_id,route_dir
0,0139b1253130b33adcd4b3a4490530d2,0a2a817a-d35d-434d-b039-6a78d79d7602,91673676-969b-4c0a-a233-158c98c522dc_0
1,0139b1253130b33adcd4b3a4490530d2,0a2a817a-d35d-434d-b039-6a78d79d7602,T6_0
2,0139b1253130b33adcd4b3a4490530d2,52c2636c-34a3-434c-99ae-cdf3dc36d15c,T11x_0


In [16]:
def get_explode_singles(
    single_route_aggregation: pd.DataFrame,
    frequency_thresholds: tuple
) -> pd.DataFrame:
    ms_precursor_threshold = min(frequency_thresholds)
    single_qual = (single_route_aggregation.query('am_max_trips_hr >= @ms_precursor_threshold & pm_max_trips_hr >= @ms_precursor_threshold')
                   .explode('route_dir')
                   .sort_values(['schedule_gtfs_dataset_key','stop_id', 'route_dir'])[['schedule_gtfs_dataset_key','stop_id', 'route_dir']]
                  )
    return single_qual

In [17]:
singles_explode = get_explode_singles(max_arrivals_by_stop_single, (HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD)).explode('route_dir')

In [18]:
from tqdm import tqdm
tqdm.pandas()

In [19]:
share_counts = {}
singles_explode.groupby(['schedule_gtfs_dataset_key', 'stop_id']).progress_apply(casf.accumulate_share_count, share_counts=share_counts)

100%|██████████| 19059/19059 [00:03<00:00, 5600.35it/s]


""


In [20]:
qualify_dict = {key: share_counts[key] for key in share_counts.keys() if share_counts[key] >= SHARED_STOP_THRESHOLD}

In [21]:
feeds_to_filter = np.unique([key.split('__')[0] for key in qualify_dict.keys()])

In [49]:
# qualify_dict

In [23]:
(singles_explode.query('route_dir.isin(["20-13191_1", "720-13191_1"])').stop_id.value_counts() == 1).value_counts()
#  True: unshared stop count False: shared stop count

True     60
False    16
Name: stop_id, dtype: int64

* local/rapid overlap, there are many unshared stops but that does not offer riders meaningful choice (still a spatial subset)
* need a spatial methodology here...

In [24]:
feeds_to_filter = np.unique([key.split('__')[0] for key in qualify_dict.keys()])

In [25]:
feeds_to_filter

array(['0666caf3ec1ecc96b74f4477ee4bc939',
       '3364ec074ca85001da3abd78be2ae521',
       '3c275e5acf8974e1afd765bd3011424c',
       '4b317fc27dde351e12253d46cedd8df0',
       '7cc0cb1871dfd558f11a2885c145d144',
       'c499f905e33929a641f083dad55c521e',
       'ecd018ad66f497fb8f188ed5a71b284b',
       'fb467982dcc77a7f9199bebe709bb700'], dtype='<U32')

In [26]:
helpers.import_scheduled_shapes?

Signature:
helpers.import_scheduled_shapes(
    analysis_date: str,
    filters: tuple = None,
    columns: list = ['shape_array_key', 'geometry'],
    get_pandas: bool = True,
    crs: str = 'EPSG:3310',
) -> Union[geopandas.geodataframe.GeoDataFrame, dask_geopandas.core.GeoDataFrame]
Docstring: Import shapes.
File:      ~/data-analyses/rt_segment_speeds/segment_speed_utils/helpers.py
Type:      function

In [27]:
## TODO integrate into script

def get_trips_with_route_dir(analysis_date):
    trips = helpers.import_scheduled_trips(
    analysis_date,
    columns = ["feed_key", "gtfs_dataset_key", "trip_id",
               "route_id", "direction_id", "route_type",
              "shape_array_key", "route_short_name", "name"],
    get_pandas = True
    )
    trips = trips[trips['route_type'].isin(['3', '11'])] #  bus only

    trips.direction_id = trips.direction_id.fillna(0).astype(int).astype(str)
    trips['route_dir'] = trips[['route_id', 'direction_id']].agg('_'.join, axis=1)
    
    return trips

In [28]:
shapes = helpers.import_scheduled_shapes(analysis_date, columns=['shape_array_key', 'geometry'])

trips = (get_trips_with_route_dir(analysis_date)
         .query("schedule_gtfs_dataset_key.isin(@feeds_to_filter)")
         .drop_duplicates(subset=['schedule_gtfs_dataset_key', 'shape_array_key', 'route_dir'])
        
        )

In [29]:
trips.head(3)

,feed_key,schedule_gtfs_dataset_key,trip_id,route_id,direction_id,route_type,shape_array_key,route_short_name,name,route_dir
1671,9e8eed3cc17c4fdde369dd07bcb45faa,3c275e5acf8974e1afd765bd3011424c,11273689|25712684:T5|10:36:00,11273689,0,3,bcd4f310183437ac0b20a2eb6426c0d5,None,Stanford Schedule,11273689_0
1686,9e8eed3cc17c4fdde369dd07bcb45faa,3c275e5acf8974e1afd765bd3011424c,11273689|25817216:T11|12:14:00,11273689,1,3,78a42d5496baf388fc40fa027f76d3f0,None,Stanford Schedule,11273689_1
1700,9e8eed3cc17c4fdde369dd07bcb45faa,3c275e5acf8974e1afd765bd3011424c,14602095|25712661:T4|16:07:00,14602095,1,3,97bea6ce476801b6e456c0e88b3cfba6,RP PM,Stanford Schedule,14602095_1


In [30]:
trips.drop_duplicates(subset=['name'])

,feed_key,schedule_gtfs_dataset_key,trip_id,route_id,direction_id,route_type,shape_array_key,route_short_name,name,route_dir
1671,9e8eed3cc17c4fdde369dd07bcb45faa,3c275e5acf8974e1afd765bd3011424c,11273689|25712684:T5|10:36:00,11273689,0,3,bcd4f310183437ac0b20a2eb6426c0d5,None,Stanford Schedule,11273689_0
3466,f6774d861953d4f4cdcffec95e2652c7,4b317fc27dde351e12253d46cedd8df0,450090,1,1,3,1e993477fc281851b7ab7c2ee3c87cb7,1,Culver City Schedule,1_1
7348,8510daa6c8576e648fcbd4f92ea73a51,3364ec074ca85001da3abd78be2ae521,17536579,AIR,1,3,bf88d35e133e57ca36ae69c981aca5d9,None,San Diego Schedule,AIR_1
12323,35702a19aac0ed4d2a616627483d3850,c499f905e33929a641f083dad55c521e,8735020,10,1,3,43789e566d08d8ee428d834c6c07c529,10,Bay Area 511 AC Transit Schedule,10_1
17323,98a10e85e7dfe3ff1caa761d1bc34606,fb467982dcc77a7f9199bebe709bb700,3749742,Express 101,0,3,c16a64094bac37f9057d868c39cd006f,Express 101,Bay Area 511 Santa Clara Transit Schedule,Express 101_0
19236,a0024a6c86c0039081d5354b929ee347,7cc0cb1871dfd558f11a2885c145d144,11708336_M21,1,0,3,f817420b11a48d033253443b372eac68,1,Bay Area 511 Muni Schedule,1_0
35181,8d9623a1823a27925b7e2f00e44fc5bb,0666caf3ec1ecc96b74f4477ee4bc939,10010007510813-DEC24,10-13191,1,3,f45cb04e61e982437431a7d384e70a9c,10/48,LA Metro Bus Schedule,10-13191_1
91781,cd299184726656597ae2cdb4f4e81e4a,ecd018ad66f497fb8f188ed5a71b284b,12070891,1,0,3,27128e796c0bbcee787e24b5281dfbe8,1,OCTA Schedule,1_0


* dissolve shapes by route+dir, pick longest
* also try  dissolve shapes by route+dir, dissolve
* find non-overlap by previously qualifying pair

In [31]:
shapes = shapes.merge(trips, on='shape_array_key')
shapes = shapes.assign(length = shapes.geometry.length)

In [32]:
OVERLAY_BUFFER = 20
AREA_MULTIPLIER = OVERLAY_BUFFER * 2

TARGET_METERS_DIFFERENCE = 5000 #  5km per route
TARGET_AREA_DIFFERENCE = TARGET_METERS_DIFFERENCE * AREA_MULTIPLIER

In [33]:
shapes.geometry = shapes.buffer(OVERLAY_BUFFER)
shapes = shapes.assign(area = shapes.geometry.map(lambda x: x.area))

In [34]:
max_by_route_dir = shapes.groupby(['schedule_gtfs_dataset_key', 'route_dir']).length.max().reset_index()

In [35]:
shapes = (shapes.merge(max_by_route_dir, on = ['schedule_gtfs_dataset_key', 'route_dir', 'length'])
          .drop_duplicates(subset = ['schedule_gtfs_dataset_key', 'route_dir', 'length'])
         )

In [36]:
# shapes.explore(column='length')

In [37]:
feeds_to_filter

array(['0666caf3ec1ecc96b74f4477ee4bc939',
       '3364ec074ca85001da3abd78be2ae521',
       '3c275e5acf8974e1afd765bd3011424c',
       '4b317fc27dde351e12253d46cedd8df0',
       '7cc0cb1871dfd558f11a2885c145d144',
       'c499f905e33929a641f083dad55c521e',
       'ecd018ad66f497fb8f188ed5a71b284b',
       'fb467982dcc77a7f9199bebe709bb700'], dtype='<U32')

In [181]:
# gtfs_dataset_key = feeds_to_filter[0]
gtfs_dataset_key = feeds_to_filter[0]
gtfs_dataset_key

'0666caf3ec1ecc96b74f4477ee4bc939'

In [196]:
def evaluate_overlaps(gtfs_dataset_key: str, show_map: bool = False) -> list:

    this_feed_qual = {key.split(gtfs_dataset_key)[1][2:]:qualify_dict[key] for key in qualify_dict.keys() if key.split('__')[0] == gtfs_dataset_key}
    qualify_pairs = [tuple(key.split('__')) for key in this_feed_qual.keys()]

    qualify_sets = [set(x) for x in qualify_pairs]
    qualify_sets = set(map(frozenset, qualify_sets))

    unique_qualify_pairs_possible = [list(x) for x in qualify_sets]

    ## TODO accumulate pairs after check...
    unique_qualify_pairs = []
    for pair in unique_qualify_pairs_possible:
        print(f'{pair}...', end='')
        these_shapes = shapes.query('route_dir.isin(@pair) & schedule_gtfs_dataset_key == @gtfs_dataset_key')
        first_row = these_shapes.iloc[0:1][['schedule_gtfs_dataset_key', 'route_dir', 'shape_array_key', 'geometry']]
        sym_diff = first_row.overlay(these_shapes.iloc[1:2][['route_dir', 'geometry']], how='symmetric_difference')
        sym_diff = sym_diff.assign(area = sym_diff.geometry.map(lambda x: x.area),
                              route_dir = sym_diff.route_dir_1.fillna(sym_diff.route_dir_2))
        diff_area = sym_diff.area.sum()
        area_ratios = (sym_diff.area / TARGET_AREA_DIFFERENCE)
        if (sym_diff.area > TARGET_AREA_DIFFERENCE).all():
            print(f'passed, {area_ratios[0]:.2f} and {area_ratios[1]:.2f} times area target')
            m = these_shapes.explore(color='gray', tiles='CartoDB Positron')
            if show_map: display(sym_diff.explore(column='route_dir', m=m, tiles='CartoDB Positron'))
            unique_qualify_pairs += [pair]
        else:
            print(f'failed, {area_ratios[0]:.2f} and {area_ratios[1]:.2f} times area target')
            if show_map: display(these_shapes.explore(column='route_dir', tiles='CartoDB Positron'))
            
    return unique_qualify_pairs

In [200]:
evaluate_overlaps('0666caf3ec1ecc96b74f4477ee4bc939', show_map=False)

['20-13191_0', '720-13191_0']...failed, 0.03 and 0.28 times area target
['70-13191_0', '78-13191_0']...passed, 4.02 and 4.18 times area target
['53-13191_0', '55-13191_0']...passed, 3.50 and 4.96 times area target
['180-13191_0', '217-13191_0']...passed, 2.75 and 3.01 times area target
['78-13191_1', '14-13191_1']...passed, 4.80 and 4.71 times area target
['40-13191_0', '111-13191_0']...passed, 5.49 and 5.72 times area target
['152-13191_0', '224-13191_1']...passed, 5.05 and 5.71 times area target
['165-13191_0', '164-13191_0']...passed, 6.07 and 5.85 times area target
['2-13191_0', '4-13191_0']...passed, 5.53 and 6.30 times area target
['94-13191_0', '90-13191_0']...passed, 3.86 and 7.11 times area target
['45-13191_1', '40-13191_1']...passed, 2.96 and 5.32 times area target
['76-13191_1', '70-13191_1']...passed, 4.47 and 4.36 times area target
['152-13191_1', '224-13191_0']...passed, 5.73 and 5.13 times area target
['60-13191_1', '55-13191_1']...passed, 6.66 and 3.58 times area targe

[['70-13191_0', '78-13191_0'],
 ['53-13191_0', '55-13191_0'],
 ['180-13191_0', '217-13191_0'],
 ['78-13191_1', '14-13191_1'],
 ['40-13191_0', '111-13191_0'],
 ['152-13191_0', '224-13191_1'],
 ['165-13191_0', '164-13191_0'],
 ['2-13191_0', '4-13191_0'],
 ['94-13191_0', '90-13191_0'],
 ['45-13191_1', '40-13191_1'],
 ['76-13191_1', '70-13191_1'],
 ['152-13191_1', '224-13191_0'],
 ['60-13191_1', '55-13191_1'],
 ['40-13191_0', '210-13191_0'],
 ['60-13191_1', '251-13191_1'],
 ['53-13191_0', '18-13191_1'],
 ['40-13191_1', '111-13191_1'],
 ['76-13191_1', '78-13191_1'],
 ['260-13191_0', '70-13191_0'],
 ['70-13191_0', '76-13191_0'],
 ['76-13191_0', '78-13191_0'],
 ['251-13191_0', '60-13191_0'],
 ['4-13191_1', '2-13191_1'],
 ['76-13191_1', '14-13191_1'],
 ['164-13191_1', '165-13191_1'],
 ['70-13191_1', '260-13191_1'],
 ['210-13191_1', '40-13191_1'],
 ['180-13191_1', '217-13191_1'],
 ['94-13191_1', '217-13191_1'],
 ['70-13191_1', '78-13191_1'],
 ['81-13191_0', '94-13191_0'],
 ['40-13191_0', '45-13

## Adding stops

In [190]:
max_arrivals_by_stop_single.head(3)

,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count
0,0139b1253130b33adcd4b3a4490530d2,00eb15cb-1430-4964-b8ae-ca6183e1d0ef,2,[D1_0],4.0,0.67,1.00,6.0,1
1,0139b1253130b33adcd4b3a4490530d2,02a30e39-496f-45d4-ba1c-ac8f3c66b621,2,[0ad6c6aa-1939-45a0-a3a8-02ebe8e19092_0],6.0,0.67,1.50,8.0,1
2,0139b1253130b33adcd4b3a4490530d2,02a30e39-496f-45d4-ba1c-ac8f3c66b621,2,[D2_0],3.0,0.67,0.75,5.0,1


In [192]:
def find_stops_this_pair(feed_stops: pd.DataFrame, one_feed_pair: list) -> pd.DataFrame:
    feed_stops = (feed_stops.explode(column='route_dir')
                  .query('route_dir in @one_feed_pair')
                  .groupby(['schedule_gtfs_dataset_key', 'stop_id'])[['route_dir']]
                  .count()
                  .reset_index()
                 )
    return feed_stops.query('route_dir > 1')

In [ ]:
def find_stops_this_feed(gtfs_dataset_key: str,
                         max_arrivals_by_stop_single: pd.DataFrame,
                         unique_qualify_pairs: list) -> pd.DataFrame:
    
    feed_stops = max_arrivals_by_stop_single.query('schedule_gtfs_dataset_key == @gtfs_dataset_key')
    stop_dfs = []
    for pair in unique_qualify_pairs:
        stop_dfs += [find_stops_this_pair(metro_stops, pair)]    

    feed_add = pd.concat(stop_dfs).merge(feeds, on = 'schedule_gtfs_dataset_key')
    return feed_add
    feed_add = stops.merge(feed_add, on = ['feed_key', 'stop_id'])

## LBT check

That location is at the intersection of 2nd and PCH, Stop name: PCH & 2nd NE, stop ID #1465.

The Long Beach Transit 121 has 20 minute frequencies almost all day. route-121 - ridelbt
The Long Beach Transit 171 has 20 minute frequencies at peak hours (morning and evening). routes-171-175 - ridelbt

Under AB2553, this should qualify that stop as being a major transit stop.

-> frequency actually insufficient with averaging.

In [43]:
stops = helpers.import_scheduled_stops(analysis_date,
                                      columns=['feed_key', 'stop_id', 'geometry'])

In [44]:
trips = get_trips_with_route_dir(analysis_date)

In [63]:
feeds = trips[['feed_key', 'schedule_gtfs_dataset_key', 'name']].drop_duplicates()

In [46]:
lbt_feed = feeds.query('name.str.contains("Long Beach")')

In [47]:
max_stops = stops.merge(lbt_feed, on = 'feed_key').merge(max_arrivals_by_stop_single, on = ['schedule_gtfs_dataset_key', 'stop_id'])

In [48]:
# max_stops.explode('route_dir').query('~route_dir.str.contains("131")').explore()